# Microbial Community Disease Risk Prediction

In this case study, we develop and evaluate neural network models to predict disease risk from high-dimensional microbiome sequence data. Our dataset consists of 16S rRNA gene profiles from 16,344 stool samples, with approximately half of the samples coming from individuals diagnosed with type 1 diabetes (cases) and the other half from individuals without a diagnosis of type 1 diabetes (controls). By modeling patterns in these microbial community profiles, our goal is to accurately classify samples based on disease status. This could provide insight into whether an individual's gut microbiome composition is associated with their risk of developing type 1 diabetes. We implement and compare several neural network architectures, including convolutional and recurrent networks, to evaluate their ability to capture predictive signals from large-scale marker gene surveys of complex microbial communities.

In [5]:
import pandas
dataframe = pandas.read_csv('https://raw.githubusercontent.com/bryankolaczkowski/ALS3200C/main/mbiome.data.csv')
dataframe.head()

,DTA0,DTA1,DTA2,DTA3,DTA4,DTA5,DTA6,DTA7,DTA8,DTA9,...,DTA247,DTA248,DTA249,DTA250,DTA251,DTA252,DTA253,DTA254,DTA255,LBL0
0,1.92,1.80,1.44,1.79,1.68,1.42,1.52,1.58,1.43,1.45,...,-2.02,-2.32,-2.19,-2.25,-2.25,-2.29,-2.19,-2.63,-2.86,1.0
1,1.97,1.98,2.16,2.12,1.78,1.71,1.69,1.60,1.74,1.64,...,-2.05,-1.97,-1.92,-2.12,-1.94,-2.18,-2.45,-2.63,-2.87,0.0
2,2.25,2.11,2.05,1.92,2.08,1.93,1.87,1.57,1.81,1.61,...,-2.02,-1.87,-1.95,-2.09,-1.96,-1.99,-2.01,-2.57,-2.71,1.0
3,2.25,2.07,1.92,1.84,1.83,1.80,1.88,1.48,1.70,1.46,...,-1.94,-2.11,-2.22,-1.98,-2.22,-2.00,-2.10,-2.59,-2.84,0.0
4,2.28,2.27,2.26,2.20,2.01,2.00,1.99,1.92,1.68,1.79,...,-1.69,-1.66,-1.82,-1.88,-1.92,-1.89,-2.07,-2.50,-2.72,0.0


The dataset contains 256 columns labeled DTA0 through DTA255, representing the relative abundances of 256 bacterial taxa across the 16,344 samples. Relative abundance is calculated as the proportion of sequencing reads assigned to each taxon in a given sample. To account for sequencing depth differences, the relative abundances have been normalized using a center log-ratio transformation - a standard technique for compositional microbial data. This transforms the data to a log scale centered at zero, where positive values indicate higher than average abundance, and negative values indicate lower than average abundance for a given taxon. After transformation, the relative abundance values typically range from +2.5 to -2.5.

The target prediction variable is contained in the LBL0 column, with 0 indicating control samples and 1 indicating case samples positive for type 1 diabetes. Our modeling goal is to accurately predict this disease label based on the multivariate microbial community profiles in columns DTA0 through DTA255.

We first split the data into training and validation subsets for model fitting and evaluation. We then extract the DTA columns as model inputs and the LBL0 column as labels. To leverage TensorFlow's sequence modeling capabilities, we expand the data to three dimensions, with the additional dimension representing the ordered sequence of taxa.

In the following sections, we fit and compare several neural network architectures, including convolutional and recurrent networks, to evaluate their ability to capture predictive signals about disease status from the full set of microbial taxa abundances and their co-occurrence patterns.

In [6]:
import numpy as np

# create train-validate split
train_dataframe = dataframe.sample(frac=0.8, random_state=2100963)
valid_dataframe = dataframe.drop(train_dataframe.index)
print(train_dataframe.shape, valid_dataframe.shape, dataframe.shape)

# extract explanatory variables
dta_ids = [ x for x in dataframe.columns if x.find('DTA') == 0 ]
train_x = np.expand_dims(train_dataframe[dta_ids].to_numpy(), axis=-1)
valid_x = np.expand_dims(valid_dataframe[dta_ids].to_numpy(), axis=-1)
print(train_x.shape, valid_x.shape)

# extract labels
train_y = train_dataframe['LBL0'].to_numpy()
valid_y = valid_dataframe['LBL0'].to_numpy()
print(train_y.shape, valid_y.shape)

(13075, 257) (3269, 257) (16344, 257)
(13075, 256, 1) (3269, 256, 1)
(13075,) (3269,)


We can see that there are 16,344 total samples in our dataframe. We've extracted 13,057 for training and 3,269 for validation.

After expanding the data dimension, we have explanatory variables of shape (256,1), a one-dimensional sequence of 256 bacterial 'species'.

## simple linear model

Now that we have some training and validation data, we just need to build a classifier to predict disease risk from the data.

First, we'll start with a simple linear model implemented using a single Dense neuron with sigmoid output, as this is a binary classification problem.

In previous cases, we were able to send the data directly to the Dense layer. But in this case, because we have 'expanded' the last dimension of the data - in order to fit tensorflow's sequence models - we need to 'collapse' that dimension back down, so it can be properly analyzed by the Dense layer.

It's pretty easy to do this; we just need to use a Flatten layer to 'flatten' the 'expanded' data back down to a simple vector. And, because the Flatten layer is just like any other tensorflow Layer object, we can use it as the *first* layer in our network, provided we set the input_shape option.

The following code cell implements a simple linear classifier for our disease-risk prediction problem.

In [4]:
import pandas
import numpy as np
import tensorflow as tf

# download data
dataframe = pandas.read_csv('https://raw.githubusercontent.com/bryankolaczkowski/ALS3200C/main/mbiome.data.csv')

# create train-validate split
train_dataframe = dataframe.sample(frac=0.8, random_state=2100963)
valid_dataframe = dataframe.drop(train_dataframe.index)

# extract explanatory variables
dta_ids = [ x for x in dataframe.columns if x.find('DTA') == 0 ]
train_x = np.expand_dims(train_dataframe[dta_ids].to_numpy(), axis=-1)
valid_x = np.expand_dims(valid_dataframe[dta_ids].to_numpy(), axis=-1)

# extract labels
train_y = train_dataframe['LBL0'].to_numpy()
valid_y = valid_dataframe['LBL0'].to_numpy()

# package data into tensorflow dataset
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(32)
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y)).batch(32)

# build model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(256,1)))
model.add(tf.keras.layers.Dense(units=1, activation=tf.keras.activations.sigmoid))
model.summary()

# compile model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

# fit model
model.fit(train_data, epochs=20, validation_data=valid_data)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 257 (1.00 KB)
Trainable params: 257 (1.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/20
409/409 [==============================] - 2s 4ms/step - loss: 0.6910 - accuracy: 0.5468 - val_loss: 0.6895 - val_accuracy: 0.5216
Epoch 2/20
409/409 [==============================] - 2s 6ms/step - loss: 0.6686 - accuracy: 0.6028 - val_loss: 0.6711 - val_accuracy: 0.5418
Epoch 3/20
409/409 [==============================] - 1s 3ms/step - loss: 0.6516 - accuracy: 0.6481 - val_loss: 0.6547 - val_accuracy: 0

Our initial baseline model is a simple linear classifier with only 257 trainable parameters - 256 input weights plus a single bias term. Despite its simplicity, it achieves solid performance, reaching approximately 80% training accuracy and 78% validation accuracy after 20 epochs. This indicates that a substantial amount of predictive signal for the disease labels is contained in a linear combination of the microbial taxa abundances. The linear model provides a reasonable baseline to benchmark more complex nonlinear models against. We now explore whether convolutional and recurrent neural network architectures can capture higher-order nonlinear relationships and covariation in the community structure to improve classification accuracy beyond this linear baseline.

## convolution model

In the next code section, we implement a basic one-layer convolutional neural network for disease classification. The convolution layer consists of 16 filters of length 3 applied with same padding, such that the output sequence length remains 256. We use a tanh activation, which is less common than ReLU for modern convnets but allows for more direct comparison to the default activation in recurrent networks. With 4,161 trainable weights, this nonlinear model has substantially more parameters than the linear baseline. To account for this increased model capacity, we train for 50 epochs which is longer than the linear model. This convolution network allows us to model localized nonlinear interactions among adjacent taxa in the community profiles. We can evaluate whether directly capturing these short-range taxon co-occurrence patterns improves predictive performance beyond the linear model.

In [3]:
import pandas
import numpy as np
import tensorflow as tf

# download data
dataframe = pandas.read_csv('https://raw.githubusercontent.com/bryankolaczkowski/ALS3200C/main/mbiome.data.csv')

# create train-validate split
train_dataframe = dataframe.sample(frac=0.8, random_state=2100963)
valid_dataframe = dataframe.drop(train_dataframe.index)

# extract explanatory variables
dta_ids = [ x for x in dataframe.columns if x.find('DTA') == 0 ]
train_x = np.expand_dims(train_dataframe[dta_ids].to_numpy(), axis=-1)
valid_x = np.expand_dims(valid_dataframe[dta_ids].to_numpy(), axis=-1)

# extract labels
train_y = train_dataframe['LBL0'].to_numpy()
valid_y = valid_dataframe['LBL0'].to_numpy()

# package data into tensorflow dataset
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(32)
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y)).batch(32)

# build model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv1D(filters=16,
                                 kernel_size=(3,),
                                 activation=tf.keras.activations.tanh,
                                 padding='same',
                                 input_shape=(256,1)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=1, activation=tf.keras.activations.sigmoid))
model.summary()

# compile model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

# fit model
model.fit(train_data, epochs=50, validation_data=valid_data)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 256, 16)           64        
                                                                 
 flatten_2 (Flatten)         (None, 4096)              0         
                                                                 
 dense_4 (Dense)             (None, 1)                 4097      
                                                                 
Total params: 4161 (16.25 KB)
Trainable params: 4161 (16.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Epoch 1/50
409/409 [==============================] - 1s 1ms/step - loss: 0.6752 - accuracy: 0.5778 - val_loss: 0.6363 - val_accuracy: 0.7167
Epoch 2/50
409/409 [==============================] - 0s 1ms/step - loss: 0.6129 - accuracy: 0.6747 - val_loss: 0.5996 - val_accuracy: 0.6455
Epoch 3/50
409/409 [==============================] - 1s 2ms/step - loss: 0.5505 - accuracy: 0.7339 - val_loss: 0.5484 - val_accuracy: 0.7222
Epoch 4/50
409/409 [==============================] - 0s 1ms/step - loss: 0.5100 - accuracy: 0.7623 - val_loss: 0.5200 - val_accuracy: 0.7485
Epoch 5/50
409/409 [==============================] - 0s 1ms/step - loss: 0.4852 - accuracy: 0.7798 - val_loss: 0.4999 - val_accuracy: 0.7589
Epoch 6/50
409/409 [==============================] - 0s 1ms/step - loss: 0.4663 - accuracy: 0.7920 - val_loss: 0.4843 - val_accuracy: 0.7645
Epoch 7/50
409/409 [==============================] - 0s 1ms/step - loss: 0.4516 - accuracy: 0.7998 - val_loss: 0.4722 - val_accuracy: 0.7687
Epoch 

## LSTM model

In [4]:
import pandas
import numpy as np
import tensorflow as tf

# download data
dataframe = pandas.read_csv('https://raw.githubusercontent.com/bryankolaczkowski/ALS3200C/main/mbiome.data.csv')

# create train-validate split
train_dataframe = dataframe.sample(frac=0.8, random_state=2100963)
valid_dataframe = dataframe.drop(train_dataframe.index)

# extract explanatory variables
dta_ids = [ x for x in dataframe.columns if x.find('DTA') == 0 ]
train_x = np.expand_dims(train_dataframe[dta_ids].to_numpy(), axis=-1)
valid_x = np.expand_dims(valid_dataframe[dta_ids].to_numpy(), axis=-1)

# extract labels
train_y = train_dataframe['LBL0'].to_numpy()
valid_y = valid_dataframe['LBL0'].to_numpy()

# package data into tensorflow dataset
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(32)
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y)).batch(32)

# build model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(units=16, return_sequences=True, input_shape=(256,1)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=1, activation=tf.keras.activations.sigmoid))
model.summary()


# compile model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

# fit model
model.fit(train_data, epochs=50, validation_data=valid_data)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 256, 16)           1152      
                                                                 
 flatten_3 (Flatten)         (None, 4096)              0         
                                                                 
 dense_5 (Dense)             (None, 1)                 4097      
                                                                 
Total params: 5249 (20.50 KB)
Trainable params: 5249 (20.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Epoch 1/50
409/409 [==============================] - 11s 25ms/step - loss: 0.6791 - accuracy: 0.5606 - val_loss: 0.6420 - val_accuracy: 0.7054
Epoch 2/50
409/409 [==============================] - 10s 25ms/step - loss: 0.6165 - accuracy: 0.6766 - val_loss: 0.6013 - val_accuracy: 0.6601
Epoch 3/50
409/409 [==============================] - 11s 27ms/step - loss: 0.5451 - accuracy: 0.7407 - val_loss: 0.5271 - val_accuracy: 0.7443
Epoch 4/50
409/409 [==============================] - 11s 26ms/step - loss: 0.4920 - accuracy: 0.7725 - val_loss: 0.5017 - val_accuracy: 0.7489
Epoch 5/50
409/409 [==============================] - 10s 26ms/step - loss: 0.4624 - accuracy: 0.7846 - val_loss: 0.4892 - val_accuracy: 0.7525
Epoch 6/50
409/409 [==============================] - 10s 25ms/step - loss: 0.4431 - accuracy: 0.7966 - val_loss: 0.4839 - val_accuracy: 0.7571
Epoch 7/50
409/409 [==============================] - 10s 25ms/step - loss: 0.4292 - accuracy: 0.8054 - val_loss: 0.4808 - val_accuracy: